In [1]:
import keras


Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4497778049515494309
]


In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Add,Concatenate, Dropout,Conv2D
from keras.models import Model

MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 100
#VALIDATION_SPLIT = 0.2


In [4]:
def clean_str(string):
    """
    Cleaning of dataset
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [6]:
import os 
os.chdir("C:\\Users\\rohan\\Desktop\\LeadingIndia.ai\\TASK3")
df1= pd.read_csv('pt1.csv')
df2= pd.read_csv('pt2.csv')
#data_valid = pd.read_csv("valid.tsv", sep='\t')
#data_test=pd.read_csv("test.tsv",sep='\t')
#data_train.head()

#data_train.columns = ['id','label','statement','subject','speaker', 'job', 'state','party','barely_true_c','false_c','half_true_c','mostly_true_c','pants_on_fire_c','venue']
 
#data_valid.columns =['id','label','statement','subject','speaker', 'job', 'state','party','barely_true_c','false_c','half_true_c','mostly_true_c','pants_on_fire_c','venue']



#data_test = ['id','label','statement','subject','speaker', 'job', 'state','party','barely_true_c','false_c','half_true_c','mostly_true_c','pants_on_fire_c','venue']


#df = pd.concat([data_train, data_valid])

data_train=df2.merge(df1, left_on='id', right_on='id')
data_train.head(5)
data_train.to_csv('data.csv')
#data_train.columns
#data_train['Label']
#data_train['Label']


#data_train=pd.read_csv("data.csv")
#data_train["Label"]
data_train.head(20)

,id,text,label
0,7.311660e+17,ðŸ”¥ca kkk grand wizard ðŸ”¥ endorses @hillary...,unverified
1,7.145990e+17,an open letter to trump voters from his top st...,unverified
2,6.918090e+17,america is a nation of second chances â€”@potu...,non-rumor
3,6.932050e+17,"brandon marshall visits and offers advice, sup...",non-rumor
4,5.511000e+17,rip elly may clampett: so sad to learn #beverl...,true
5,7.672240e+17,former 3 doors down guitarist matt roberts has...,non-rumor
6,7.155160e+17,craigslist ad: â€˜get paid $15 an hour to prot...,unverified
7,5.141060e+17,just in: missing afghan soldiers found trying ...,true
8,5.003200e+17,"the day #ferguson cops told a dirty, bloody li...",unverified
9,5.003200e+17,"the day #ferguson cops told a dirty, bloody li...",unverified


In [7]:

# Input Data preprocessing

print(data_train.columns)
print('What the raw input data looks like:')
print(data_train[0:5])
print(data_train.shape)
#data_train = data_train.drop(['Unnamed: 0'],axis = 1)
print(data_train.shape[0])

texts = []
labels = []

for i in range(data_train.shape[0]):
    text1 = data_train.id[i]
    text2 = data_train.text[i]
    
    text = str(text1) +""+ str(text2) 
    texts.append(text)
    if data_train.label[i] == 'false' :
        labels.append(0)
    elif data_train.label[i] == 'true' :
        labels.append(1)
    elif data_train.label[i] == 'unverified' : 
        labels.append(2)
    elif data_train.label[i] == 'non-rumor' : 
        labels.append(3)
    
  




Index(['id', 'text', 'label'], dtype='object')
What the raw input data looks like:
             id                                               text       label
0  7.311660e+17  ðŸ”¥ca kkk grand wizard ðŸ”¥ endorses @hillary...  unverified
1  7.145990e+17  an open letter to trump voters from his top st...  unverified
2  6.918090e+17  america is a nation of second chances â€”@potu...   non-rumor
3  6.932050e+17  brandon marshall visits and offers advice, sup...   non-rumor
4  5.511000e+17  rip elly may clampett: so sad to learn #beverl...        true
(1714, 3)
1714


In [8]:
print(labels)
print(len(labels))
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

[2, 2, 3, 3, 1, 3, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 1, 2, 2, 1, 2, 2, 1, 3, 2, 0, 3, 1, 0, 3, 2, 0, 2, 2, 2, 0, 3, 0, 0, 3, 1, 1, 1, 1, 0, 2, 3, 2, 0, 3, 3, 3, 3, 2, 2, 2, 2, 2, 0, 2, 2, 3, 2, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 2, 2, 2, 3, 3, 0, 1, 1, 1, 1, 0, 2, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 2, 3, 1, 0, 1, 2, 1, 1, 2, 3, 3, 0, 2, 2, 3, 1, 1, 3, 0, 2, 2, 3, 0, 0, 3, 3, 2, 2, 2, 2, 3, 0, 3, 0, 3, 2, 2, 0, 1, 2, 2, 1, 1, 3, 0, 0, 0, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 1, 3, 2, 2, 1, 1, 1, 1, 0, 0, 2, 2, 3, 1, 1, 1, 1, 1, 3, 0, 3, 3, 3, 1, 1, 1, 1, 2, 3, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 0, 0, 1, 0, 1, 2, 1, 1, 1, 0, 2, 2, 2, 3, 1, 2, 1, 2, 3, 3, 1, 2, 1, 1, 1, 2, 1, 0, 2, 1, 2, 3, 1, 1, 1, 1, 0, 2, 0, 1, 3, 1, 1, 0, 0, 0, 2, 3, 0, 0, 3, 0, 3, 3, 3, 3, 3, 0, 1, 2, 2, 2, 2, 3, 3, 2, 2, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 3, 2, 3, 3, 1, 0, 0, 2, 0, 2, 0, 3, 0, 0, 1, 0, 3, 1, 

In [10]:
# Pad input sequences
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels),num_classes = 4)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (1714, 100)
Shape of label tensor: (1714, 6, 4)


In [11]:

# Train test validation Split
from sklearn.model_selection import train_test_split

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
x_train, x_test, y_train, y_test = train_test_split( data, labels, test_size=0.05, random_state=42)
x_test, x_val, y_test, y_val = train_test_split( data, labels, test_size=0.6, random_state=42)
print('Size of train, validation, test:', len(y_train), len(y_val), len(y_test))

print('real & fake news in train,valt,test:')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))
print(y_test.sum(axis=0))

Size of train, validation, test: 1628 1029 685
real & fake news in train,valt,test:
[[1247.  381.    0.    0.]
 [1141.  487.    0.    0.]
 [1250.  378.    0.    0.]
 [1246.  382.    0.    0.]
 [1628.    0.    0.    0.]
 [1628.    0.    0.    0.]]
[[ 791.  238.    0.    0.]
 [ 717.  312.    0.    0.]
 [ 796.  233.    0.    0.]
 [ 783.  246.    0.    0.]
 [1029.    0.    0.    0.]
 [1029.    0.    0.    0.]]
[[526. 159.   0.   0.]
 [489. 196.   0.   0.]
 [511. 174.   0.   0.]
 [529. 156.   0.   0.]
 [685.   0.   0.   0.]
 [685.   0.   0.   0.]]


In [12]:
#Using Pre-trained word embeddings 
os.chdir("C:\\Users\\rohan\\Desktop\\LeadingIndia.ai")
GLOVE_DIR = "data" 
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    #print(values[1:])
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove.' % len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH)

Total 400000 word vectors in Glove.


In [13]:
# Simple CNN model
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu',padding='same',kernel_regularizer='l2')(embedded_sequences)
l_pool1 = MaxPooling1D(5,padding='same')(l_cov1)

l_flat = Flatten()(l_pool1)
l_dense = Dense(128, activation='relu',kernel_regularizer='l2')(l_flat)
preds = Dense(6, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("Fitting the simple convolutional neural network model")
model.summary()
history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=32)


Fitting the simple convolutional neural network model
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 100)          699700    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 128)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2560)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               327808    
_________________________________________________________________
dense

ValueError: Error when checking target: expected dense_2 to have 2 dimensions, but got array with shape (1628, 6, 4)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
    
l_merge = Concatenate(axis=1)(convs)
l_cov1= Conv1D(filters=128, kernel_size=5, activation='relu',padding='same')(l_merge)
l_pool1 = MaxPooling1D(5,padding='same')(l_cov1)
l_cov2 = Conv1D(filters=128, kernel_size=5, activation='relu',padding='same')(l_pool1)
l_pool2 = MaxPooling1D(30,padding='same')(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(6, activation='softmax')(l_dense)

model2 = Model(sequence_input, preds)
model2.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['acc'])

print("Fitting a more complex convolutional neural network model")
model2.summary()
history2 = model2.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=3, batch_size=32)
model2.save('model.h5')

In [ ]:
# list all data in history
print(history2.history.keys())
import matplotlib.pyplot as plt
%matplotlib inline 
# summarize history for accuracy
plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
# Test model 1
test_preds = model.predict(x_test)
test_preds = np.round(test_preds)
correct_predictions = float(sum(test_preds == y_test)[0])
print("Correct predictions:", correct_predictions)
print("Total number of test examples:", len(y_test))
print("Accuracy of model1: ", correct_predictions/float(len(y_test)))

# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
x_pred = model.predict(x_test)
x_pred = np.round(x_pred)
x_pred = x_pred.argmax(1)
y_test_s = y_test.argmax(1)
cm = confusion_matrix(y_test_s, x_pred)
plt.matshow(cm, cmap=plt.cm.binary, interpolation='nearest')
plt.title('Confusion matrix - model1')
plt.colorbar()
plt.ylabel('expected label')
plt.xlabel('predicted label')
# plt.show()

#Test model 2
test_preds2 = model2.predict(x_test)
test_preds2 = np.round(test_preds2)
correct_predictions = float(sum(test_preds2 == y_test)[0])
print("Correct predictions:", correct_predictions)
print("Total number of test examples:", len(y_test))
print("Accuracy of model2: ", correct_predictions/float(len(y_test)))

# Creating the Confusion Matrix
x_pred = model2.predict(x_test)
x_pred = np.round(x_pred)
x_pred = x_pred.argmax(1)
y_test_s = y_test.argmax(1)
cm = confusion_matrix(y_test_s, x_pred)
plt.matshow(cm, cmap=plt.cm.binary, interpolation='nearest',)
plt.title('Confusion matrix - model2')
plt.colorbar()
plt.ylabel('expected label')
plt.xlabel('predicted label')
plt.show()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
score = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])